In [1]:
#dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from pandas import json_normalize
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import json

def chicago_crime_to_sql():
    # json url to pandas df
    api_key="pk.eyJ1IjoiZ2VvcmdpYWZiaSIsImEiOiJja25na20zcmwwN2l2MnFwa3BrZ2JmaHdxIn0.5aZk4GmJsmPQMAgpdia8lQ"
    url = 'https://data.cityofchicago.org/resource/qzdf-xmn8.json'
    # reverseGeoAPI = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{lng}%2C%20{lt}.json?access_token={API_KEY}"

    df = pd.read_json(url)
    df.drop_duplicates("id", inplace=True)

    new_df =df.set_index("id")
    new_df.dropna(inplace = True)
    new_df = new_df.drop(['x_coordinate','y_coordinate','updated_on', 'beat', 'iucr'],axis=1)
    new_df['latitude'] = pd.to_numeric(new_df['latitude'], errors = 'coerce')
    new_df["zip_code"]=""

    for index, row in new_df.iterrows():
        lt=row['latitude']
        lng=row['longitude']
        crime=row['primary_type']
    #     date=pd.to_datetime(row['date']).strftime("%m/%d/%Y %H:%M:%S")
        # Run a request to endpoint and convert result to json
        target_url= f"https://api.mapbox.com/geocoding/v5/mapbox.places/{lng}%2C%20{lt}.json?access_token={api_key}"
        geo_data = requests.get(target_url).json()
        new_df.loc[index,'zip_code']=geo_data['features'][2]['text']
    #     new_df.loc[index,'date']=date


    new_df['zip_code'] = pd.to_numeric(new_df['zip_code'], errors = 'coerce')
    # new_df['zip_code']=new_df['zip_code'].astype(int)
    new_df.dropna(inplace=True)
    new_df["zip_code"]=new_df.zip_code.astype(int)

    new_df=new_df.drop(['location'],axis=1)

    user_name = 'postgres'
    pword = 'Baggins89'
    connection_string = f"postgres:{pword}@localhost:5432/crime_db"
    engine = create_engine(f'postgresql://{connection_string}')

    new_df.to_sql(name='chicago',con=engine,if_exists='replace',index=True)

    sqlite_engine=create_engine('sqlite:///chicago_crime_table_schema.sqlite')
    new_df.to_sql(name='chicago',con=sqlite_engine,if_exists='replace',index=True)
    
    engine.dispose()
    sqlite_engine.dispose()
    

In [2]:
# Create engine using the `demographics.sqlite` database file
def chicago_crime_dict():
    user_name = 'postgres'
    pword = 'Baggins89'
    connection_string = f"postgres:{pword}@localhost:5432/crime_db"
    connected_engine=create_engine(f'postgresql://{connection_string}').connect()

    chicago_crime_df=pd.read_sql_table('chicago',connected_engine).set_index('id')


    json_result=chicago_crime_df.to_json(orient="table")
    parsed=json.loads(json_result)
    return parsed['data']

In [3]:
print(chicago_crime_dict())

[{'id': 12262588, 'date': '2020-12-31T23:59:00.000Z', 'district': 25, 'block': '017XX N CICERO AVE', 'latitude': 41.9116839, 'description': 'OVER $500', 'location_description': 'PARKING LOT / GARAGE (NON RESIDENTIAL)', 'community_area': 25, 'ward': 37, 'case_number': 'JE104819', 'year': 2020, 'domestic': False, 'fbi_code': '06', 'longitude': -87.745917518, 'primary_type': 'THEFT', 'arrest': False, 'zip_code': 60639}, {'id': 12258658, 'date': '2020-12-31T23:51:00.000Z', 'district': 10, 'block': '013XX S HOMAN AVE', 'latitude': 41.863406877, 'description': 'AGGRAVATED - OTHER DANGEROUS WEAPON', 'location_description': 'STREET', 'community_area': 29, 'ward': 24, 'case_number': 'JE100080', 'year': 2020, 'domestic': False, 'fbi_code': '04B', 'longitude': -87.710455868, 'primary_type': 'BATTERY', 'arrest': False, 'zip_code': 60623}, {'id': 12259515, 'date': '2020-12-31T23:45:00.000Z', 'district': 16, 'block': '048XX W PATTERSON AVE', 'latitude': 41.947024331, 'description': 'OVER $500', 'loc